# ADMM 算法

### ADMM 算法

ADMM 可以用来求解形如
$$\begin{align*}
\min_{x,z}\  & f(x)+g(z)\\
\mathrm{s.t.}\  & Ax+Bz=c
\end{align*}$$
的优化问题，其中 $f$ 和 $g$ 是凸函数。

ADMM 的迭代公式为
$$
\begin{align*}
x^{k+1} & =\underset{x}{\arg\min}\ f(x)+\frac{\rho}{2}\Vert Ax+Bz^{k}-c+u^{k}\Vert^{2}\\
z^{k+1} & =\underset{z}{\arg\min}\ g(z)+\frac{\rho}{2}\Vert Ax^{k+1}+Bz-c+u^{k}\Vert^{2}\\
u^{k+1} & =u^{k}+Ax^{k+1}+Bz^{k+1}-c.
\end{align*}
$$

定义原问题残差 $r^{k+1}=Ax^{k+1}+Bz^{k+1}-c$ 和对偶问题残差 $s^{k+1}=\rho A'B(z^{k+1}-z^{k})$。当 $||r^k||$ 和 $||s^k||$ 小于某个阈值时即可认为算法收敛。

### Lasso

Lasso 是一种带有变量选择效果的回归方法，它与线性回归中的最小二乘方法（OLS）类似，但加上了对回归系数的 $L^1$ 范数惩罚项。为了与 ADMM 算法的记号匹配，我们用 $M\in\mathbb{R}^{n\times p}$ 表示自变量矩阵，$b\in\mathbb{R}^n$ 表示因变量向量，要估计的回归系数为 $x\in\mathbb{R}^p$。于是 Lasso 的目标函数为 $$\frac{1}{2}\Vert Mx-b\Vert^2+\lambda \Vert x\Vert_1,$$ 其中 $\Vert v\Vert_1$ 表示向量 $v=(v_1,\ldots,v_n)'$ 的 $L^1$ 范数，即 $\Vert v\Vert_1=|v_1|+\cdots+|v_n|$。

Lasso 可以改写为 ADMM 的形式：$f(x)=(1/2)\cdot\Vert Mx-b\Vert^2$，$g(z)=\lambda ||z||_1$，约束中 $A=I_p$, $B=-I_p$，$c=0$。其迭代公式为

$$
\begin{align*}
x^{k+1} & =(M'M+\rho I)^{-1}(M'b+\rho(z^{k}-u^{k}))\\
z^{k+1} & =S_{\lambda/\rho}(x^{k+1}+u^{k})\\
u^{k+1} & =u^{k}+x^{k+1}-z^{k+1},
\end{align*}
$$

其中 $S_{\kappa}(a)$ 为 soft-thresholding 运算符，定义为

$$
S_{\kappa}(a)=\begin{cases}
a-\kappa, & a>\kappa\\
0, & |a|\le\kappa\\
a+\kappa, & a<-\kappa
\end{cases},
$$

一种紧凑的表达是 $S_{\kappa}(a)=\mathrm{sign}(a)\cdot\max\{0,|a|-\kappa\}$。

相应地，原问题残差为 $r^{k+1}=x^{k+1}-z^{k+1}$，对偶问题残差为 $s^{k+1}=-\rho (z^{k+1}-z^{k})$。

### 利用 ADMM 求解 Lasso

In [17]:
import numpy as np
np.set_printoptions(linewidth=100)

生成模拟数据：

In [18]:
np.random.seed(123)
n = 1000
p = 30
nz = 10
M = np.random.normal(size=(n, p))
# 真实的 x 只有前10个元素非零，其余均为0
xtrue = np.random.normal(size=nz)
xtrue = np.concatenate((xtrue, np.zeros(p - nz)))
b = M.dot(xtrue) + np.random.normal(size=n)
xtrue

array([-1.05417044, -0.78301134,  1.82790084,  1.7468072 ,  1.3282585 , -0.43277314, -0.6686141 ,
       -0.47208845,  1.05554064,  0.67905585,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

将 $\lambda$ 设为 $\lambda=0.01n$，编写 ADMM 算法来对 Lasso 问题进行求解。

**注意** ：注意到在每一次迭代中都要计算 $(M'M+\rho I)^{-1}v$，其中 $v$ 是某个向量。如果直接使用 `np.linalg.solve()`，计算量会非常大。一种更好的方法是先对 $M'M+\rho I$ 进行 Cholesky 分解（$M'M+\rho I$ 是正定矩阵），然后再解线性方程组。

In [19]:
def soft_thresholding(a, k):
    return np.sign(a) * np.maximum(0.0, np.abs(a) - k)

from scipy.linalg import cho_factor, cho_solve

# initialization
rho = 1.0
lam = 0.01*n
A = np.ones(p)
B = -A
MtM  = M.transpose().dot(M)
Mtb = M.transpose().dot(b)
I = np.ones(p)

c, lower = cho_factor(MtM+rho*I)

x = np.zeros(p)
z = np.zeros(p)
u = np.zeros(p)

max_iter = 10000

kappa = lam / rho



Lasso 可以改写为 ADMM 的形式：$f(x)=(1/2)\cdot\Vert Mx-b\Vert^2$，$g(z)=\lambda ||z||_1$，约束中 $A=I_p$, $B=-I_p$，$c=0$。其迭代公式为

$$
\begin{align*}
x^{k+1} & =(M'M+\rho I)^{-1}(M'b+\rho(z^{k}-u^{k}))\\
z^{k+1} & =S_{\lambda/\rho}(x^{k+1}+u^{k})\\
u^{k+1} & =u^{k}+x^{k+1}-z^{k+1},
\end{align*}
$$

其中 $S_{\kappa}(a)$ 为 soft-thresholding 运算符，定义为

$$
S_{\kappa}(a)=\begin{cases}
a-\kappa, & a>\kappa\\
0, & |a|\le\kappa\\
a+\kappa, & a<-\kappa
\end{cases},
$$

一种紧凑的表达是 $S_{\kappa}(a)=\mathrm{sign}(a)\cdot\max\{0,|a|-\kappa\}$。

相应地，原问题残差为 $r^{k+1}=x^{k+1}-z^{k+1}$，对偶问题残差为 $s^{k+1}=-\rho (z^{k+1}-z^{k})$。

In [22]:
np.random.seed(123)
n = 1000
p = 30
nz = 10
M = np.random.normal(size=(n, p))
# 真实的 x 只有前10个元素非零，其余均为0
xtrue = np.random.normal(size=nz)
xtrue = np.concatenate((xtrue, np.zeros(p - nz)))
b = M.dot(xtrue) + np.random.normal(size=n)

def soft_thresholding(a, k):
    return np.sign(a) * np.maximum(0.0, np.abs(a) - k)

from scipy.linalg import cho_factor, cho_solve

# initialization
rho = 1.0
lam = 0.01*n
A = np.ones(p)
B = -A
MtM  = M.transpose().dot(M)
Mtb = M.transpose().dot(b)
I = np.ones(p)

c, lower = cho_factor(MtM+rho*I)

x = np.zeros(p)
z = np.zeros(p)
u = np.zeros(p)

max_iter = 10000

kappa = lam / rho


tol = 0.01
# iteration

resid_r = -999
resid_s = -999

for iter in range(max_iter):
    xnew = cho_solve((c,lower),Mtb+rho*(z-u))
    znew = soft_thresholding(xnew+u,kappa)
    unew = u + xnew - znew
    
    resid_r = np.linalg.norm(xnew-znew)
    resid_s = - rho*np.linalg.norm(znew-z)

    x = xnew
    z = znew
    u = unew

    # 打印残差信息，判断是否收敛
    if iter % 100 == 0:
        print(f"Iteration {iter}, ||r|| = {resid_r:.6f}, ||s|| = {resid_s:.6f}")
    if resid_r <= tol and resid_s <= tol:
        break

    

    print(x)

    print(xtrue)


Iteration 0, ||r|| = 3.520730, ||s|| = -0.000000
[-1.13160133 -0.78822161  1.81566129  1.76626825  1.31550137 -0.41480033 -0.63158673 -0.49471122
  1.01642951  0.69186113 -0.00701014 -0.0425267   0.02809037  0.00257502  0.0327561  -0.02353521
  0.03400501 -0.05075188  0.0095235   0.01743599  0.04690838  0.02074242  0.02245351  0.07137934
 -0.03628979  0.02394227  0.00318267 -0.02400846  0.0131833  -0.00772103]
[-1.05417044 -0.78301134  1.82790084  1.7468072   1.3282585  -0.43277314 -0.6686141  -0.47208845
  1.05554064  0.67905585  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
[-1.13036046 -0.78757849  1.81386944  1.76452931  1.31423238 -0.41435064 -0.63108    -0.49437686
  1.01547223  0.69120735 -0.00698037 -0.04255878  0.0282928   0.00280695  0.03290058 -0.0235218
  0.03417234 -0.05080669  0.00957232  0.0175